<a href="https://colab.research.google.com/github/epogrebnyak/pychain/blob/main/escrow_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from dataclasses import dataclass
from uuid import uuid4
from typing import Dict, TypedDict, List
from collections import UserDict

Amount = int
Address = str

def uuid(digits=5) -> Address:
  """Return short UUID (like 'az4s'). Shortened for readability."""
  return str(uuid4())[:digits]

class Contract:
  pass

@dataclass
class Content:
  balance: Amount = 0
  contract: Contract | None = None

def genesis() -> "Chain":
  """Return empty chain ready to be filled."""
  return Chain(data={})

@dataclass
class Chain:
   data: Dict[Address, Content]

   def __getitem__(self, key: str) -> Content:
      return self.data[key]

   def add_wallet(self, balance=0) -> Address:
      address = uuid()
      self.data[address] = Content(balance, None)
      return address

   def add_contract(self, contract: Contract) -> Address:
      address = uuid()
      self.data[address] = Content(0, contract)
      return address

   def addresses(self):
      return list(self.data.keys())

   def transfer(self, sender: Address, receiver: Address, amount: Amount):
      assert self.data[sender].balance >= amount
      self.data[receiver].balance += amount
      self.data[sender].balance -= amount

@dataclass
class Escrow(Contract):
  payer: Address
  deposits: Dict[Address, Amount]
  allow_withdraw: bool = False

  def transfer(self, sender: Address, receiver: Address, amount: Amount):
      assert(sender in self.deposits.keys())
      assert(amount <= self.deposits[sender])
      self.deposits[sender] = self.deposits[sender] - amount
      # reciever may be a new address, not previoiusly in self.deposits
      self.deposits[receiver] = self.deposits.get(receiver, 0) + amount
      return True

  def release_funds(self, payer: Address):
      # release funds logic can be extended, right now everything is at
      # discretion of payer
      if payer == self.payer:
         self.allow_withdraw = True
         return True
      return False

  # FIXME: not the greatest way that we have to provide 'this_contract' here,
  #        consider it a temp fix.
  def withdraw(self, chain: Chain, this_contract: Address, payee: Address):
      assert(payee in self.deposits.keys())
      amount = self.deposits[payee]
      chain.transfer(this_contract, payee, amount)
      self.deposits[payee] = 0
      return True

class ChainError(ValueError):
      pass

def assert_has_funds(chain, address, amount) -> bool:
      if chain[address].balance >= amount:
          return True
      raise ChainError(f"Insufficient funds at {address}, cannot transfer {amount}.")

def create_escrow(chain: Chain, sender: Address, contractor: Address, amount: Address) -> Escrow:
      assert_has_funds(chain, sender, amount)
      escrow = Escrow(payer=sender, deposits={contractor: amount})
      contract_address = chain.add_contract(escrow)
      chain.transfer(sender, contract_address, amount)
      return contract_address

In [3]:
from typing import Callable, Tuple

def maxlen(xs: List[str]) -> int:
    return max(map(len, xs))

@dataclass
class Column:
  header: str
  items: List[str]

  @property
  def width(self):
    return max(maxlen(self.items), len(self.header))

  @property
  def underline(self):
    """Header underline string."""
    return "-" * self.width

  def printable(self) -> "PrintableColumn":
    return PrintableColumn([self.header, self.underline] + self.items, self.width)

def align_right(width, strings):
  return [s.strip().rjust(width) for s in strings]

def align_left(width, strings):
  return [s.strip().ljust(width) for s in strings]

@dataclass
class PrintableColumn:
  items: List[str]
  width: int

  def align_left(self):
    return PrintableColumn(align_left(self.width, self.items), self.width)

  def align_right(self):
    return PrintableColumn(align_right(self.width, self.items), self.width)

  def __iter__(self):
        yield from self.items

def column(header, items):
  return Column(header, list(items)).printable().align_left()

def show_contract(content: Content) -> str:
    if (k := content.contract) is None:
       return ""
    return str(k)

@dataclass
class Table:
  columns: List[Column]

  def purge(self):
    last_column = self.columns[-1]
    last_column.items[1] = "-" * len(last_column.items[0].strip())
    return self

  def print(self):
    for row in zip(*self.purge().columns):
      print(*row, sep=" | ")

@dataclass
class Viewer:
   chain: Chain
   who_is_who: Dict
   headers: Tuple[str] = ("UUID", "Role", "Balance", "Contract")

   def print(self):
      data = self.chain.data
      values = data.values()
      addresses = column(self.headers[0], data.keys())
      names = column(self.headers[1],
                     [self.who_is_who.get(address, "") for address in data.keys()]
                     )
      amounts = column(self.headers[2], [str(v.balance) for v in values])
      contracts = column(self.headers[3], [show_contract(v) for v in values])
      t = Table([addresses, names, amounts.align_right(), contracts])
      t.print()

In [10]:
column("abc", ["-----"])

PrintableColumn(items=['abc  ', '-----', '-----'], width=5)

In [5]:
Table([PrintableColumn(["abc", "-----", "text"], 5).align_right().align_left()]).print()


abc  
---
text 


In [6]:
"abc".rjust(5), "{:>5}".format("abc")


('  abc', '  abc')

In [9]:
# 0. Создали условный блокчейн, в котором записываются величины денег и контракты.
chain = genesis()

# 1. У всех есть пустые кошельки и только у заказчика есть 500 тыс. денежных единиц
payer_address = chain.add_wallet(balance=500_000)
contractor_address = chain.add_wallet(0)
subcontractor_address = chain.add_wallet(0)
employee_address = chain.add_wallet(0)
who_is_who = dict([(payer_address, "Заказчик"), (contractor_address, "Исполнитель"),
                   (subcontractor_address, "Субподрядчик"), (employee_address, "Работник")])
print("\n1. В начале нашего примера деньги есть только у заказчика.")
viewer = Viewer(chain, who_is_who, headers=("Адрес", "Описание", "Остаток", "Контракт"))
viewer.print()

# 2. Открываем счет эскроу, заказчик перечисляет на него деньги.
escrow_address = create_escrow(chain, payer_address, contractor_address, 500_000)
who_is_who[escrow_address] = "Контракт эскроу"
print("\n2. Заказчик перевел деньги на счет эксроу.")
viewer.print()

# 3. Переводим деньги внутри экроу, сумма на кошельках не меняется.
escrow = chain[escrow_address].contract
escrow.transfer(contractor_address, subcontractor_address, 160_000)
escrow.transfer(contractor_address, employee_address, 40_000)
print("\n3. Исполнители провели расчеты внутри эскроу, поменялся контракт эскроу.")
viewer.print()

# 4. Раскрываем эскроу по акту работ, переводим деньги исполнителям.
escrow.release_funds(payer_address)
print("\n4. Заказчик раскрыл аккредитив, поменялся контракт эскроу.")
viewer.print()

# 5. Исполнители забирают средства.
escrow.withdraw(chain, escrow_address, contractor_address)
escrow.withdraw(chain, escrow_address, subcontractor_address)
escrow.withdraw(chain, escrow_address, employee_address)
print("\n5. Исполнители забрали средства из эскроу, расчеты закончены.")
viewer.print()


1. В начале нашего примера деньги есть только у заказчика.
Адрес | Описание     | Остаток | Контракт
----- | ------------ | ------- | --------
cc7fc | Заказчик     |  500000 |         
189f3 | Исполнитель  |       0 |         
392bf | Субподрядчик |       0 |         
8c8c8 | Работник     |       0 |         

2. Заказчик перевел деньги на счет эксроу.
Адрес | Описание        | Остаток | Контракт                                                               
----- | --------------- | ------- | --------
cc7fc | Заказчик        |       0 |                                                                        
189f3 | Исполнитель     |       0 |                                                                        
392bf | Субподрядчик    |       0 |                                                                        
8c8c8 | Работник        |       0 |                                                                        
f70ff | Контракт эскроу |  500000 | Escrow(payer='cc7fc', d